In [102]:
import pandas as pd
import numpy as np
from math import log
from __future__ import division

In [103]:
df = pd.read_csv('TSC_churn_telecom.txt')
var_disc = [v for v in df.columns if v[:2]=='C_']
var_cont = [v for v in df.columns if v[:2]=='V_']
obj = ['TARGET']
ID = ['ID']
df.head()

,V_ACCT_LGHT,C_IP,C_VMP,V_N_VM_MESS,V_T_DAY_CHG,V_T_EVE_CALLS,V_T_EVE_CHG,V_T_NIG_CALLS,V_T_NIG_CHG,V_T_INT_CALLS,V_T_INT_CHG,V_CS_CALLS,TARGET,ID
0,128,No,Yes,25,45.07,99,16.78,91,11.01,3,2.70,1,0,1
1,107,No,Yes,26,27.47,103,16.62,103,11.45,3,3.70,1,0,2
2,137,No,No,0,41.38,110,10.30,104,7.32,5,3.29,0,0,3
3,65,No,No,0,21.95,83,19.42,111,9.40,6,3.43,4,1,11
4,168,No,No,0,21.90,71,8.92,128,6.35,2,3.02,1,0,13


## WoE para variables continuas

In [104]:
def WoEfc(X, var, obj, ID, bins):
    Y = X.copy()
    IV = []
    for i in bins:
        Y[var] = pd.cut(X[var], bins = i)
        a = Y.pivot_table(aggfunc='count',columns=obj,fill_value=0, index=var,values=ID[0]).reset_index()
        a.loc[-1] =["TOTAL", sum(a[0]), sum(a[1])]
        IV.append(0)
        for y in set(Y[var].values):
            goods = int(a[a[var]==y][1])/int(a[a[var]=='TOTAL'][1])
            bads = a[a[var]==y][0]/int(a[a[var]=='TOTAL'][0])
            d = log(bads/goods)
            IV[-1] += float((bads-goods)*d)
        print "IV con "+str(i) + " bins es " + str(IV[-1])
    armax = np.argmax(IV)
    print "Se el maximiza el IV con "+ str(bins[armax]) + " bins"
    Y[var] = pd.cut(X[var], bins = bins[armax])
    a = Y.pivot_table(aggfunc='count',columns=obj,fill_value=0, index=var,values=ID[0]).reset_index()
    a.loc[-1] =["TOTAL", sum(a[0]), sum(a[1])]
    for y in set(Y[var].values):
            goods = int(a[a[var]==y][1])/int(a[a[var]=='TOTAL'][1])
            bads = a[a[var]==y][0]/int(a[a[var]=='TOTAL'][0])
            d = log(bads/goods)
            Y[var].replace(y,d,inplace = True)
    print a 
    return Y

In [106]:
WoEfc(X=df, var='V_T_DAY_CHG', obj = obj, ID = ID, bins=[1,2,15,4,5,16,15])

IV con 1 bins es 0.0
IV con 2 bins es 0.0445922321088
IV con 15 bins es 0.807393324819
IV con 4 bins es 0.543977536547
IV con 5 bins es 0.552685256072
IV con 16 bins es 0.790092844674
IV con 15 bins es 0.807393324819
Se el maximiza el IV con 15 bins
TARGET       V_T_DAY_CHG     0    1
 0       (8.767, 11.681]    32    5
 1      (11.681, 14.553]    41    7
 2      (14.553, 17.424]    95   10
 3      (17.424, 20.295]   144   20
 4      (20.295, 23.167]   179   32
 5      (23.167, 26.038]   274   22
 6      (26.038, 28.909]   306   46
 7      (28.909, 31.781]   306   18
 8      (31.781, 34.652]   329   20
 9      (34.652, 37.523]   284   18
 10     (37.523, 40.395]   202   37
 11     (40.395, 43.266]   124   37
 12     (43.266, 46.137]    67   40
 13     (46.137, 49.009]    32   41
 14      (49.009, 51.88]     7   26
-1                 TOTAL  2422  379


,V_ACCT_LGHT,C_IP,C_VMP,V_N_VM_MESS,V_T_DAY_CHG,V_T_EVE_CALLS,V_T_EVE_CHG,V_T_NIG_CALLS,V_T_NIG_CHG,V_T_INT_CALLS,V_T_INT_CHG,V_CS_CALLS,TARGET,ID
0,128,No,Yes,25,-1.339000,99,16.78,91,11.01,3,2.70,1,0,1
1,107,No,Yes,26,0.040131,103,16.62,103,11.45,3,3.70,1,0,2
2,137,No,No,0,-0.645449,110,10.30,104,7.32,5,3.29,0,0,3
3,65,No,No,0,-0.133163,83,19.42,111,9.40,6,3.43,4,1,11
4,168,No,No,0,-0.133163,71,8.92,128,6.35,2,3.02,1,0,13
5,95,No,No,0,0.040131,75,21.05,115,8.65,5,3.32,3,0,14
6,62,No,No,0,-0.133163,76,26.11,99,9.14,6,3.54,4,0,15
7,85,No,Yes,27,0.945513,90,23.88,75,4.02,4,3.73,1,0,17
8,93,No,No,0,0.945513,111,18.55,121,5.83,3,2.19,3,0,18
9,76,No,Yes,33,0.945513,65,18.09,108,7.46,5,2.70,1,0,19


In [108]:
for x in var_cont:
    df = WoEfc(X=df, var=x, obj = obj, ID = ID, bins=[1,2,15,4,5,16,15])

IV con 1 bins es 0.0
IV con 2 bins es 0.00308158744464
IV con 15 bins es 0.0438262978154
IV con 4 bins es 0.00333090013882
IV con 5 bins es 0.0064063685006
IV con 16 bins es 0.0346871442965
IV con 15 bins es 0.0438262978154
Se el maximiza el IV con 15 bins
TARGET     V_ACCT_LGHT     0    1
 0      (11.817, 24.2]    44   10
 1        (24.2, 36.4]    72    5
 2        (36.4, 48.6]   100   15
 3        (48.6, 60.8]   142   22
 4        (60.8, 73.0]   236   34
 5        (73.0, 85.2]   242   36
 6        (85.2, 97.4]   293   45
 7       (97.4, 109.6]   295   48
 8      (109.6, 121.8]   273   51
 9      (121.8, 134.0]   258   45
 10     (134.0, 146.2]   175   20
 11     (146.2, 158.4]   124   17
 12     (158.4, 170.6]    84   15
 13     (170.6, 182.8]    47   13
 14     (182.8, 195.0]    37    3
-1               TOTAL  2422  379
IV con 1 bins es 0.0
IV con 2 bins es 0.057482826312
IV con 15 bins es inf
IV con 4 bins es 0.0747456871111
IV con 5 bins es 0.108726595521
IV con 16 bins es inf
IV 

In [109]:
df.head()

,V_ACCT_LGHT,C_IP,C_VMP,V_N_VM_MESS,V_T_DAY_CHG,V_T_EVE_CALLS,V_T_EVE_CHG,V_T_NIG_CALLS,V_T_NIG_CHG,V_T_INT_CALLS,V_T_INT_CHG,V_CS_CALLS,TARGET,ID
0,-0.108516,No,Yes,2.709535,-1.339000,0.040382,0.398073,0.349123,-0.087151,0.012823,-0.014851,0.438844,0,1
1,-0.039038,No,Yes,0.091097,0.040131,0.040382,0.398073,0.094951,-0.356927,0.012823,-0.902446,0.438844,0,2
2,0.314241,No,No,-0.148329,-0.645449,0.020828,0.178109,0.094951,-0.067127,0.266359,0.084271,0.089165,0,3
3,0.082659,No,No,-0.148329,-0.133163,-0.090303,-0.124422,0.281570,-0.089545,0.124158,0.304672,-1.667214,1,11
4,-0.132046,No,No,-0.148329,-0.133163,-0.616438,0.273419,-0.488721,0.503342,-0.589704,0.216261,0.438844,0,13


## WoE para variables discretas

In [110]:
def WoEfd(X, var, obj, ID):
    Y = X.copy()
    IV = 0
    a = Y.pivot_table(aggfunc='count',columns=obj,fill_value=0, index=var,values=ID[0]).reset_index()
    a.loc[-1] =["TOTAL", sum(a[0]), sum(a[1])]
    for y in set(Y[var].values):
            goods = int(a[a[var]==y][1])/int(a[a[var]=='TOTAL'][1])
            bads = a[a[var]==y][0]/int(a[a[var]=='TOTAL'][0])
            d = log(bads/goods)
            IV += float((bads-goods)*d)
            Y[var].replace(y,d,inplace = True)
    print a 
    print "IV total de "+ str(IV)
    return Y

In [111]:
for x in var_disc:
    df = WoEfd(X=df, var=x, obj = obj, ID = ID)

TARGET   C_IP     0    1
 0         No  2260  273
 1        Yes   162  106
-1      TOTAL  2422  379
IV total de 0.359517795238
TARGET  C_VMP     0    1
 0         No  1730  314
 1        Yes   692   65
-1      TOTAL  2422  379
IV total de 0.0752320543513


In [1]:
df.head()

NameError: name 'df' is not defined